In [4]:
import pandas as pd
import arxiv
import os

In [7]:
client=arxiv.Client()

In [8]:
search = arxiv.Search(
  query = "misinformation",
  max_results = 10000,
  sort_by = arxiv.SortCriterion.SubmittedDate
)


In [15]:
results=client.results(search)

In [16]:
all_results = list(results)

KeyboardInterrupt: 

In [14]:
all_results

[]

In [ ]:
len(os.listdir('PeerRead/data/arxiv.cs.lg_2007-2017/train/parsed_pdfs'))

In [67]:
all_paper=[]
for paper in os.listdir('PeerRead/data/arxiv.cs.cl_2007-2017/train/parsed_pdfs'):
    data=pd.read_json('PeerRead/data/arxiv.cs.cl_2007-2017/train/parsed_pdfs/' + paper).T.reset_index()
    data=data.iloc[1:]
    all_paper.append(data)

    # break

In [68]:
dataset=pd.concat(all_paper).reset_index(drop=True)

In [ ]:
dataset.columns

In [ ]:
dataset['authors']

In [71]:
dataset.dropna(inplace=True)

In [ ]:
dataset['title']

In [74]:
dataset['title']=dataset['title'].str.lower()

In [ ]:
dataset[dataset['title'].str.contains('information')]